In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gzip
import tarfile
import glob
import os
import sklearn
import numpy
import pandas as pd
from sklearn.utils import resample
import keras
!pip install keras_metrics
import keras_metrics
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM, Dense, Embedding
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score, classification_report

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/CIS 530 Project/Milestone 3/train_data.csv').dropna().drop_duplicates()
val_data = pd.read_csv('/content/drive/MyDrive/CIS 530 Project/Milestone 3/val_data.csv').dropna().drop_duplicates()
test_data = pd.read_csv('/content/drive/MyDrive/CIS 530 Project/Milestone 3/test_data.csv').dropna().drop_duplicates()
datasets = [train_data, val_data, test_data]

In [ ]:
# Downsample non-controversial to balance classes
train_non_data = train_data[train_data['contro'] == 0]
train_con_data = train_data[train_data['contro'] == 1]
train_non_balanced = resample(train_non_data, replace=False, n_samples = len(train_con_data))
combined = pd.concat([train_con_data, train_non_balanced])
train_data = combined.sample(frac=1).reset_index(drop=True)
train_data = train_data.drop(columns=['Unnamed: 0'])

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatize_word(word):
  return lemmatizer.lemmatize(word)

words_to_ignore = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')
def clean_str(s):
  tokenized_words = tokenizer.tokenize(s)
  tokenized_words = [word for word in tokenized_words if word not in words_to_ignore]
  tokenized_words = [lemmatize_word(word) for word in tokenized_words]
  return ' '.join(tokenized_words)

In [ ]:
for dataset in datasets:
  dataset['comment_text'] = dataset['comment_text'].apply(lambda text: clean_str(text))

In [ ]:
max_features = 50000
keras_tokenizer = Tokenizer(num_words=max_features)
keras_tokenizer.fit_on_texts(train_data['comment_text'].to_list())
tokenized_training_data = keras_tokenizer.texts_to_sequences(train_data['comment_text'].to_list())
tokenized_validation_data = keras_tokenizer.texts_to_sequences(val_data['comment_text'].to_list())
tokenized_testing_data = keras_tokenizer.texts_to_sequences(test_data['comment_text'].to_list())
train_labels = numpy.array(train_data['contro'].to_list())
val_labels = numpy.array(val_data['contro'].to_list())
test_labels = numpy.array(test_data['contro'].to_list())

In [ ]:
# Padding
max_len = 300
pad_train = pad_sequences(tokenized_training_data, maxlen=max_len)
pad_val = pad_sequences(tokenized_validation_data, maxlen=max_len)
pad_test = pad_sequences(tokenized_testing_data, maxlen=max_len)

In [ ]:
# The model
embedding = 128
model = Sequential()
model.add(Embedding(max_features, embedding, input_length=max_len))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])

In [ ]:
batch_size = 128
epochs = 5
model.fit(pad_train, train_labels, epochs=epochs, validation_data=(pad_val, val_labels))

Epoch 1/5
812/812 [==============================] - 254s 271ms/step - loss: 0.4223 - accuracy: 0.8028 - precision: 0.8099 - recall: 0.7888 - val_loss: 0.2902 - val_accuracy: 0.8850 - val_precision: 0.4669 - val_recall: 0.9202
Epoch 2/5
812/812 [==============================] - 218s 268ms/step - loss: 0.1357 - accuracy: 0.9502 - precision: 0.9414 - recall: 0.9599 - val_loss: 0.3546 - val_accuracy: 0.8693 - val_precision: 0.4333 - val_recall: 0.9264
Epoch 3/5
812/812 [==============================] - 218s 268ms/step - loss: 0.0840 - accuracy: 0.9704 - precision: 0.9659 - recall: 0.9753 - val_loss: 0.2393 - val_accuracy: 0.9156 - val_precision: 0.5538 - val_recall: 0.8748
Epoch 4/5
812/812 [==============================] - 218s 268ms/step - loss: 0.0463 - accuracy: 0.9843 - precision: 0.9833 - recall: 0.9855 - val_loss: 0.5146 - val_accuracy: 0.8307 - val_precision: 0.3685 - val_recall: 0.9307
Epoch 5/5
812/812 [==============================] - 218s 268ms/step - loss: 0.0423 - accura

In [ ]:
predictions = model.predict_classes([pad_test], batch_size=128)
predictions_confidence = model.predict([pad_test], batch_size=128)
predictions = [x for [x] in predictions]
predictions_confidence = [x for [x] in predictions_confidence]
print(predictions_confidence)
print(predictions)
target_names = ['non-contro', 'contro']
print(classification_report(test_labels, predictions, target_names=target_names))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[0.00041096722, 0.00029605956, 0.00016407917, 0.00036357623, 0.026772475, 0.0002578252, 0.000508961, 0.00013001433, 0.00017410534, 0.99962926, 0.0006127479, 0.00030283508, 0.0006979995, 0.999006, 0.0004079198, 0.00038357326, 0.011163125, 0.0073582744, 0.9807811, 0.00042268934, 0.00017083889, 0.0001600625, 0.955309, 0.00046027717, 0.00033520692, 0.0012832077, 0.023151187, 0.025659863, 0.00071035617, 0.0007765701, 0.00038347088, 0.00018710947, 0.0005553979, 0.0065741455, 0.005810851, 0.0005044309, 0.9328911, 0.000585692, 0.999566, 0.0002086307, 0.0009830508, 0.00055573916, 0.00031530458, 0.0003371779, 0.9968346, 0.0006650758, 0.00047055856, 0.4560667, 0.0076042106, 0.021519354, 0.00031822274, 0.0003029015, 0.14310333, 0.0026409568, 0.00021884592, 0.0021095, 0.06913939, 0.00023607683, 0.0012778377, 0.002760157, 0.0003782762, 0.12744454, 0.0003313263, 0.99680173, 0.0016266156, 0.9975091, 0.014312005, 0.00019142, 0.00053788663, 0.00015180343, 0.9976528, 0.025268655, 0.0027525865, 0.00152733

In [ ]:
actual = test_labels
# 5 incorrect predictions on either side

fp_count = 0
# indices of FP
fp_indices = []
for i in range(len(predictions)):
  if fp_count == 100:
    break
  if predictions[i] == 1 and actual[i] == 0:
    fp_indices.append(i)
    fp_count += 1

fn_count = 0
# indices of FN
fn_indices = []
for i in range(len(predictions)):
  if fn_count == 100:
    break
  if predictions[i] == 0 and actual[i] == 1:
    fn_indices.append(i)
    fn_count += 1

fp_comments = []
fn_comments = []

for i in range(5):
  fp_comments.append(test_data.iloc[fp_indices[i]]['comment_text'])
  fn_comments.append(test_data.iloc[fn_indices[i]]['comment_text'])

print('Comments Identified as Controversial that were Actually Labeled as Non-Controversial:')
for num, comment in enumerate(fp_comments[:5]):
  print(num, comment)
print()
print('Comments Identified as Non-Controversial that were Actually Labeled as Controversial:')
for num, comment in enumerate(fn_comments[:5]):
  print(num, comment)

print(fp_indices)
print(fn_indices)

Comments Identified as Controversial that were Actually Labeled as Non-Controversial:
0 I focussing science moment leading revolution
1 Orangemike Time move collateral target I suggest User FisherQueen girl protected
2 Quoting Hoppe first Detailing Kinsella important point At diff I put Democracy section Hoppe earlier comment first followed Kinsella said e Stephan Kinsella writes Hoppe critic accused Hoppe homophobia bigotry like based passage Kinsella wrote Hoppe discussion physically removing homosexual group applied private covenant based community centered around traditional value He shared letter Hope wrote saying gay couple street mind business would expelled openly hostile basic heterosexual private property basis society As I said edit summary choose totally take comment context attack NOT mean Wikipedia used purpose talkie talkie
3 He self confessed approved observation see e g history Soviet partisan BTW first object
4 Users using sockpuppets discussion Note user nightscream 